In [14]:
import numpy as np
from scipy import stats

alpha = 0.95
t_syst = 0.001
m = {"m1":2.93, "m2":4.82, "m3":5.88}
tab = {"m1":(1.808, 1.687,1.712,1.675,1.685, 1.787,1.682, 1.718, 1.687, 1.829), "m2":(1.419, 1.405, 1.381, 1.257, 1.347, 1.416, 1.336, 1.318, 1.317, 1.361), "m3":(1.209, 1.263, 1.166, 1.260, 1.231, 1.223, 1.187, 1.189, 1.210, 1.204)}
u = len(tab["m1"])

#расчет среднего значения времени
tab_avg = {}
for n in tab:
    sum_t = 0
    for j in tab[n]:
        sum_t += j
    tab_avg[n] = round(sum_t/len(tab[n]), 3)

#расчёт абсолютной погрешности для времени
s = {}
sum_t=0
for n in tab:
    for t in tab[n]:
        sum_t = (t - tab_avg[n])**2
    s[n] = round(((sum_t/(u*(u-1))))**(0.5),3)

#Коэффициент Стьюдента
t_an = stats.t.ppf((1 + alpha)/2, u-1)

#расчёт случайно погрешности для времени
t_sl = {}
for n in s:
    t_sl[n] = t_an * s[n]

#расчёт абсолютной погрешности для времени
abs_t = {}
for e in m:
    abs_t[e] = (t_sl[e]**2 + t_syst**2)**0.5

#расчёт относительной погрешности для времени
delta_t = {}
for i in m:
    delta_t[i] = abs_t[i]/tab_avg[i]




